In [1]:
import pandas as pd
import json
import pickle
import numpy as np
from xgboost import XGBRegressor
import xgboost
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import optuna

In [2]:
print(xgboost.__version__) 

0.90


In [3]:
def cate_emb(row):
    #print("row: ", row)
    temp = pd.DataFrame()
    temp['id'] = row[1:-1].split(', ')
    temp = temp.merge(embeddings, on='id', how='inner')
    temp = temp.iloc[:, 1:]
    #print("temp: ", temp)
    return temp.mean(axis=0).tolist()

In [4]:
def city_emb(row):
    #print("row: ", row)
    temp = pd.DataFrame()
    temp['id'] = ['c' + str(row)]
    #print(temp)
    #print(temp.dtypes)
    temp = temp.merge(embeddings, on='id', how='inner')
    temp = temp.iloc[:, 1:]
    #print(temp)
    return temp.mean(axis=0).tolist()

In [179]:
#emb_train.csv
#emb_comb.csv
embeddings = pd.read_csv('/data/chaoyu/data553/emb_city.csv')
embeddings

,id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,emb_11,emb_12,emb_13,emb_14,emb_15,emb_16,emb_17,emb_18,emb_19,emb_20,emb_21,emb_22,emb_23,emb_24,emb_25,emb_26,emb_27,emb_28,emb_29,emb_30,emb_31,emb_32,emb_33,emb_34,emb_35,emb_36,emb_37,emb_38,emb_39,emb_40,emb_41,emb_42,emb_43,emb_44,emb_45,emb_46,emb_47,emb_48,emb_49,emb_50,emb_51,emb_52,emb_53,emb_54,emb_55,emb_56,emb_57,emb_58,emb_59,emb_60,emb_61,emb_62,emb_63,emb_64,emb_65,emb_66,emb_67,emb_68,emb_69,emb_70,emb_71,emb_72,emb_73,emb_74,emb_75,emb_76,emb_77,emb_78,emb_79,emb_80,emb_81,emb_82,emb_83,emb_84,emb_85,emb_86,emb_87,emb_88,emb_89,emb_90,emb_91,emb_92,emb_93,emb_94,emb_95,emb_96,emb_97,emb_98,emb_99,emb_100,emb_101,emb_102,emb_103,emb_104,emb_105,emb_106,emb_107,emb_108,emb_109,emb_110,emb_111,emb_112,emb_113,emb_114,emb_115,emb_116,emb_117,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
0,vxR_YV0atFxIxfOnF9uHjQ,0.005403,0.043263,0.023720,0.209512,-0.096993,0.162324,0.419654,-0.389664,1.025340,1.127577,-0.195348,0.065307,-0.705737,-0.944052,-0.148609,-0.453248,-0.877575,-0.465281,-0.543183,-0.197277,-0.022017,0.402050,-0.326722,-0.850001,-0.489284,0.092431,-0.058455,1.172960,-0.098330,-0.158630,0.040842,-0.900066,-0.166870,-0.051136,0.506747,0.072770,0.606570,0.336461,0.805013,-0.031466,-0.270732,-0.425155,0.686408,-0.573368,0.900607,-0.017194,-0.008275,-0.323779,-0.740318,-0.012152,0.113950,0.020127,-0.926919,0.147281,-0.640198,-0.765781,0.550704,-0.840358,0.130524,-0.379622,-0.545119,-0.395115,-0.047932,0.777859,1.926721,-0.851275,-0.331505,0.004607,-0.804593,-1.053862,-0.559583,-0.437692,-0.384294,1.071040,-0.112765,-0.675092,-0.410751,0.155893,0.001695,0.822827,-0.433310,-0.051230,-0.023858,0.029782,0.321833,1.486554,0.359710,-1.160990,-0.026548,-0.384026,-0.802518,-0.497185,-1.073361,0.076291,0.638080,-0.383536,-0.429427,-1.132393,-0.092549,-0.891065,-1.005773,0.021221,-0.585525,0.385931,-0.357949,0.252721,-0.057667,-0.295870,-0.108693,-0.001207,0.107500,-0.330064,-0.923043,0.624410,-0.307084,-0.086358,0.401462,-0.270720,1.204273,-0.642355,-0.271249,0.435386,0.251528,0.511604,-0.605337,0.968575,0.848310,0.805933
1,gTw6PENNGl68ZPUpYWP50A,0.589434,-0.330073,2.488566,-1.027159,-0.523565,-0.587760,0.617558,1.529586,-0.736585,-0.060318,0.722309,-0.593270,-1.489616,-0.796114,2.609805,-0.219287,-1.843100,1.552691,-0.153451,-0.620380,1.232951,0.537858,-0.290461,-0.582682,-0.700464,-0.789012,-0.196442,0.975737,-0.128929,-0.712380,0.103366,-1.037730,-0.322416,0.689921,-0.721962,0.775807,0.587295,1.767991,1.271987,0.552424,-0.489514,0.753904,-0.253669,-1.202935,0.265689,0.756460,-0.663210,-0.197837,0.749669,0.392787,-0.414669,-0.058896,1.762716,-1.616667,0.199372,0.447275,1.395416,1.484642,0.193532,0.485799,-0.334239,-0.040043,0.545958,-0.376825,1.133650,-0.717236,0.437941,0.921854,-0.301789,-0.230200,0.338386,-0.078894,0.125191,0.871852,-0.857007,-0.718463,-0.629986,-0.509353,-0.328732,1.405728,-1.659906,0.306801,-1.456393,1.185349,-0.404407,1.109202,1.405324,-0.659807,0.041303,-0.574772,-1.254718,0.354572,0.773839,-1.193349,-0.659109,0.495377,-0.050058,-0.243265,-0.815639,-1.081556,-0.648413,-0.595111,0.268413,0.293609,0.731719,0.021517,0.617995,-0.776101,-0.471821,0.356154,0.324816,-0.891584,1.345199,0.767996,-0.396403,1.276561,0.807968,-0.066802,-0.165346,-0.831863,-0.744082,-0.167835,0.257266,0.122475,0.192920,-0.186381,-0.689227,0.503764
2,o0p-iTC5yTBV5Yab_7es4g,0.875322,-1.717702,0.112721,0.233182,0.201375,-0.839805,-0.054490,-0.718885,0.654614,0.401681,-0.095870,-0.077615,0.872339,-0.411658,0.547341,-0.914598,-0.280906,0.259309,0.049121,-1.132490,1.679041,-0.053818,0.003105,-0.682926,-0.352246,0.961727,-0.777382,0.396943,0.306134,0.591295,-0.317104,1.772546,0.165666,0.801088,0.444201,-0.256270,0.631974,-0.369403,1.154382,-0.703609,-0.105339,1.159667,-0.113402,0.190730,0.148962,0.163141,0.358138,-0.003695,0.809033,0.923704,-0.059651,0.485466,0.171645,0.482295,0.282336,-0.292981,0.214659,0.547011,0.649625,0.69

In [6]:
embeddings.dtypes

id          object
emb_0      float64
emb_1      float64
emb_2      float64
emb_3      float64
            ...   
emb_123    float64
emb_124    float64
emb_125    float64
emb_126    float64
emb_127    float64
Length: 129, dtype: object

In [64]:
df_train = pd.read_csv('/data/chaoyu/data553/yelp_train.csv')
#business.json has stars also
df_train = df_train.rename(columns={'stars': 'target'})
df_train

,user_id,business_id,target
0,vxR_YV0atFxIxfOnF9uHjQ,gTw6PENNGl68ZPUpYWP50A,5.0
1,o0p-iTC5yTBV5Yab_7es4g,iAuOpYDfOTuzQ6OPpEiGwA,4.0
2,-qj9ouN0bzMXz1vfEslG-A,5j7BnXXvlS69uLVHrY9Upw,2.0
3,E43QxgV87Ij6KxMCHcijKw,jUYp798M93Mpcjys_TTgsQ,5.0
4,T13IBpJITI32a1k41rc-tg,3MntE_HWbNNoyiLGxywjYA,5.0
...,...,...,...
455849,Tsm8VraTp5OGyVALtUiCeQ,DXlDzOcpdUE_F21tok0fgw,5.0
455850,BrqGnby6aahIOc0N_1x0Bg,_UEQPDDiSgyYqjORWeLfJg,1.0
455851,-lh59ko3dxChBSZ9U7LfUw,KPV_FVNWkgmYh1ArVlt6kg,3.0
455852,3VPVQ4fCNF1vYayTJUxKNA,YNDxeeRUARbd8GRnscJSvg,5.0


In [17]:
df_val = pd.read_csv('/data/chaoyu/data553/yelp_val.csv')
#business.json has stars also
df_val = df_val.rename(columns={'stars': 'target'})
df_val

,user_id,business_id,target
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,5.0
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,5.0
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,4.0
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,5.0
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,3.0
...,...,...,...
142039,pA9NXgASl86RImkdBtydrA,q6-SF8zHFU1AWO70k92o1Q,2.0
142040,_eUb7UGsUoSfi9n2ieF5ow,hgWMxKhrnOUd3m5nOUBIkA,4.0
142041,cEJGXB63KhROA-XmE_jgXw,0ldxjei8v4q95fApIei3Lg,5.0
142042,Z4-V0hc51oxUdULWJOufeg,j29tuUdrfaxmGjwxHdHZPA,3.0


In [18]:
df_comb = pd.concat([df_train, df_val])
df_comb

,user_id,business_id,target
0,vxR_YV0atFxIxfOnF9uHjQ,gTw6PENNGl68ZPUpYWP50A,5.0
1,o0p-iTC5yTBV5Yab_7es4g,iAuOpYDfOTuzQ6OPpEiGwA,4.0
2,-qj9ouN0bzMXz1vfEslG-A,5j7BnXXvlS69uLVHrY9Upw,2.0
3,E43QxgV87Ij6KxMCHcijKw,jUYp798M93Mpcjys_TTgsQ,5.0
4,T13IBpJITI32a1k41rc-tg,3MntE_HWbNNoyiLGxywjYA,5.0
...,...,...,...
142039,pA9NXgASl86RImkdBtydrA,q6-SF8zHFU1AWO70k92o1Q,2.0
142040,_eUb7UGsUoSfi9n2ieF5ow,hgWMxKhrnOUd3m5nOUBIkA,4.0
142041,cEJGXB63KhROA-XmE_jgXw,0ldxjei8v4q95fApIei3Lg,5.0
142042,Z4-V0hc51oxUdULWJOufeg,j29tuUdrfaxmGjwxHdHZPA,3.0


In [241]:
df_business = pd.read_csv('/data/chaoyu/data553/business_pre_comb.csv')
df_business

,business_id,city,state,is_open,stars,review_count_b,categories,postal_code
0,tZnSodhPwNr4bzrwJ1CSbw,300,ON,0,4.0,146,"[149, 841, 775]",M6G 2L7
1,42uRhhQDzbtKrUYoBPXS8A,294,WI,1,4.5,102,"[614, 775, 81, 184, 25]",53590
2,0jtRI7hVMpQHpUVtUy4ITw,139,NV,1,4.0,242,"[92, 992, 499, 360, 25, 127, 775]",89128
3,AYL_y8ahquUW0o-cvIyLbg,55,NC,1,3.5,85,"[775, 702]",28208
4,v33d7h_Zw_I9ZSxvPdqtkw,115,NV,1,4.0,113,"[192, 772]",89011
...,...,...,...,...,...,...,...,...
24946,K9ukZjvXhDjQZnrrOPqKSw,139,NV,1,4.5,108,"[614, 81, 814, 912, 200, 549]",89109
24947,JUygwe5d_MXCCOdk0WB30g,139,NV,0,4.0,109,"[668, 49]",89109
24948,kbaXNZLUyVuWbeQxH-NNKw,115,NV,1,4.0,447,"[25, 24, 775, 78, 841]",89052
24949,iYfHHScIrm9UooevAt-pZA,115,NV,1,4.5,54,"[683, 691, 957, 687, 686]",89044


In [242]:
lbe = LabelEncoder()
df_business['state'] = lbe.fit_transform(df_business['state'])
df_business

,business_id,city,state,is_open,stars,review_count_b,categories,postal_code
0,tZnSodhPwNr4bzrwJ1CSbw,300,6,0,4.0,146,"[149, 841, 775]",M6G 2L7
1,42uRhhQDzbtKrUYoBPXS8A,294,10,1,4.5,102,"[614, 775, 81, 184, 25]",53590
2,0jtRI7hVMpQHpUVtUy4ITw,139,3,1,4.0,242,"[92, 992, 499, 360, 25, 127, 775]",89128
3,AYL_y8ahquUW0o-cvIyLbg,55,2,1,3.5,85,"[775, 702]",28208
4,v33d7h_Zw_I9ZSxvPdqtkw,115,3,1,4.0,113,"[192, 772]",89011
...,...,...,...,...,...,...,...,...
24946,K9ukZjvXhDjQZnrrOPqKSw,139,3,1,4.5,108,"[614, 81, 814, 912, 200, 549]",89109
24947,JUygwe5d_MXCCOdk0WB30g,139,3,0,4.0,109,"[668, 49]",89109
24948,kbaXNZLUyVuWbeQxH-NNKw,115,3,1,4.0,447,"[25, 24, 775, 78, 841]",89052
24949,iYfHHScIrm9UooevAt-pZA,115,3,1,4.5,54,"[683, 691, 957, 687, 686]",89044


In [243]:
df_business['postal_code'] = lbe.fit_transform(df_business['postal_code'])
df_business

,business_id,city,state,is_open,stars,review_count_b,categories,postal_code
0,tZnSodhPwNr4bzrwJ1CSbw,300,6,0,4.0,146,"[149, 841, 775]",3608
1,42uRhhQDzbtKrUYoBPXS8A,294,10,1,4.5,102,"[614, 775, 81, 184, 25]",264
2,0jtRI7hVMpQHpUVtUy4ITw,139,3,1,4.0,242,"[92, 992, 499, 360, 25, 127, 775]",345
3,AYL_y8ahquUW0o-cvIyLbg,55,2,1,3.5,85,"[775, 702]",124
4,v33d7h_Zw_I9ZSxvPdqtkw,115,3,1,4.0,113,"[192, 772]",303
...,...,...,...,...,...,...,...,...
24946,K9ukZjvXhDjQZnrrOPqKSw,139,3,1,4.5,108,"[614, 81, 814, 912, 200, 549]",331
24947,JUygwe5d_MXCCOdk0WB30g,139,3,0,4.0,109,"[668, 49]",331
24948,kbaXNZLUyVuWbeQxH-NNKw,115,3,1,4.0,447,"[25, 24, 775, 78, 841]",315
24949,iYfHHScIrm9UooevAt-pZA,115,3,1,4.5,54,"[683, 691, 957, 687, 686]",313


In [245]:
emb_city = df_business.apply(lambda row: city_emb(row['city']), axis='columns', result_type='expand')
emb_city

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,0.019304,-0.177057,0.637373,0.318220,-0.006698,-0.545345,-0.306781,-0.125858,-0.793541,-0.208935,0.773403,-0.016629,0.304182,-0.245043,-0.328879,-0.640849,0.278763,0.021803,-0.735101,0.685369,0.545151,-0.221520,0.033404,0.220473,-0.236153,-0.016998,0.250888,0.879612,-0.826165,-0.138453,-0.347213,0.421021,-0.505975,-0.160681,0.236192,-0.560439,-0.203323,-0.564576,0.750364,0.700076,-0.345232,0.375900,0.068656,-0.193011,0.198889,0.020728,0.604746,-0.064802,-0.280133,-0.749602,-0.326104,-0.037335,0.231024,0.279959,0.301655,0.608201,-0.122703,-0.447051,0.020986,0.286003,0.346501,-0.823128,-0.264620,0.176721,-0.009595,-1.135337,0.485885,0.634706,0.816923,0.349658,0.906058,0.046738,-0.294949,0.204708,-0.141854,-0.804462,0.244340,-0.087588,-0.248015,0.201480,-0.450212,0.701273,0.727225,0.873402,0.254549,-0.133012,0.646283,0.404364,-0.097851,0.210423,-0.082445,-0.140194,-0.120666,0.369968,-0.072050,-0.615781,-1.122670,-0.874121,0.030103,-0.045933,-0.214983,0.339962,0.992562,0.041973,-0.188971,-1.009116,0.590533,-0.034519,0.321591,-0.265207,0.660194,-0.728218,0.613775,0.148525,-0.363351,0.389876,-0.055561,0.522080,0.482066,-0.531625,-0.749594,-0.156278,-0.172634,0.511477,-0.145456,-0.263025,-0.241127,-0.243496
1,0.657783,-1.139428,-1.409119,0.869928,-0.280212,-1.480376,-0.526623,-0.753044,-0.489739,0.118742,2.249381,-0.189061,0.563151,-1.089075,0.320481,1.378215,-1.269444,0.788732,-0.985927,0.179574,1.469556,1.115988,-0.261896,0.871474,-0.332202,0.135305,-1.254207,-0.047412,0.333694,-0.949805,-0.959544,-0.400107,0.641010,-1.987266,1.584039,-0.889744,1.687761,0.416291,1.726106,-0.551180,-1.270366,0.206664,0.143567,-0.237116,0.212676,-0.483928,-0.058783,0.322005,-1.034182,1.591441,0.558287,1.119671,0.441360,0.740791,0.531703,-0.476730,0.859004,0.184283,-1.698118,1.158969,0.334011,-0.155870,0.181241,-1.355326,0.944059,0.733871,0.605557,-0.132718,0.458136,-0.588934,0.200724,-1.084958,-1.833617,-1.529001,1.427796,-0.541753,0.115037,1.974111,-0.449561,0.409554,-0.237489,0.091197,-0.805269,0.827393,0.620039,-0.326176,0.302011,-1.412197,-0.118017,0.198602,-0.174066,-0.242182,-0.964918,-0.372071,0.370620,-0.022238,-1.142427,-2.186307,-0.685946,-1.134455,-0.383173,0.623782,-0.189035,-0.771120,1.075651,-0.106878,1.310569,0.745047,0.415386,-0.897564,1.911584,-0.547720,0.700422,-0.766923,-0.960457,0.648755,-0.661207,0.450936,-0.321271,-1.627941,-1.272138,0.215602,-1.274385,1.365814,0.952868,-1.032050,-0.966148,0.823783
2,-0.634442,0.420432,0.739058,0.198880,0.015705,-0.086093,0.539176,-0.348183,-0.003284,0.531084,0.205506,0.215019,0.294355,-0.228493,-0.707892,-0.050808,0.167726,0.043423,0.243021,-0.081663,0.632595,0.685672,-0.210689,-0.569129,-0.313465,0.541878,0.517369,0.063066,0.391540,-0.516054,-0.466807,-0.008593,-0.213822,0.172640,-0.598365,-0.091997,0.261117,-0.058103,0.437048,-0.063614,-0.325048,0.631982,-0.117357,-0.255994,0.580079,0.129825,0.269634,-0.751864,-0.301767,-0.105304,-0.039608,0.182716,0.493973,0.089085,-1.006380,0.131974,0.312660,0.416625,0.034224,-0.481170,-0.047243,-0.044834,0.157908,-0.107799,0.363141,-0.513590,0.136616,0.718180,-0.032788,-0.019250,-0.487209,0.475690,-0.802734,0.362784,-0.249663,0.404294,0.350576,0.379714,-0.544623,0.501438,-0.318808,0.163615,-0.250724,0.230335,0.232690,0.176269,0.658890,-0.541861,0.403756,0.341645,-0.510878,-0.550795,-0.354686,-0.380084,0.013607,-0.362148,0.562692,-0.628297,0.136794,0.109379,-0.384234,0.496721,0.621719,-0.585337,-0.188469,-0.151168,-0.294974,0.059944,-0.060419,-0.083219,0.287020,0.104160,0.556262,0.315660,-0.195357,0.499980,0.292784,-0.372742,0.070888,-0.450589,0.085365,0.39512

In [246]:
emb_city_name = {}
for i in range(128):
    emb_city_name[i] = 'emb_city_' + str(i)
emb_city_name

{0: 'emb_city_0',
 1: 'emb_city_1',
 2: 'emb_city_2',
 3: 'emb_city_3',
 4: 'emb_city_4',
 5: 'emb_city_5',
 6: 'emb_city_6',
 7: 'emb_city_7',
 8: 'emb_city_8',
 9: 'emb_city_9',
 10: 'emb_city_10',
 11: 'emb_city_11',
 12: 'emb_city_12',
 13: 'emb_city_13',
 14: 'emb_city_14',
 15: 'emb_city_15',
 16: 'emb_city_16',
 17: 'emb_city_17',
 18: 'emb_city_18',
 19: 'emb_city_19',
 20: 'emb_city_20',
 21: 'emb_city_21',
 22: 'emb_city_22',
 23: 'emb_city_23',
 24: 'emb_city_24',
 25: 'emb_city_25',
 26: 'emb_city_26',
 27: 'emb_city_27',
 28: 'emb_city_28',
 29: 'emb_city_29',
 30: 'emb_city_30',
 31: 'emb_city_31',
 32: 'emb_city_32',
 33: 'emb_city_33',
 34: 'emb_city_34',
 35: 'emb_city_35',
 36: 'emb_city_36',
 37: 'emb_city_37',
 38: 'emb_city_38',
 39: 'emb_city_39',
 40: 'emb_city_40',
 41: 'emb_city_41',
 42: 'emb_city_42',
 43: 'emb_city_43',
 44: 'emb_city_44',
 45: 'emb_city_45',
 46: 'emb_city_46',
 47: 'emb_city_47',
 48: 'emb_city_48',
 49: 'emb_city_49',
 50: 'emb_city_50',


In [247]:
emb_city = emb_city.rename(columns=emb_city_name)
emb_city

,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb_city_81,emb_city_82,emb_city_83,emb_city_84,emb_city_85,emb_city_86,emb_city_87,emb_city_88,emb_city_89,emb_city_90,emb_city_91,emb_city_92,emb_city_93,emb_city_94,emb_city_95,emb_city_96,emb_city_97,emb_city_98,emb_city_99,emb_city_100,emb_city_101,emb_city_102,emb_city_103,emb_city_104,emb_city_105,emb_city_106,emb_city_107,emb_city_108,emb_city_109,emb_city_110,emb_city_111,emb_city_112,emb_city_113,emb_city_114,emb_city_115,emb_city_116,emb_city_117,emb_city_118,emb_city_119,emb_city_120,emb_city_121,emb_city_122,emb_city_123,emb_city_124,emb_city_125,emb_city_126,emb_city_127
0,0.019304,-0.177057,0.637373,0.318220,-0.006698,-0.545345,-0.306781,-0.125858,-0.793541,-0.208935,0.773403,-0.016629,0.304182,-0.245043,-0.328879,-0.640849,0.278763,0.021803,-0.735101,0.685369,0.545151,-0.221520,0.033404,0.220473,-0.236153,-0.016998,0.250888,0.879612,-0.826165,-0.138453,-0.347213,0.421021,-0.505975,-0.160681,0.236192,-0.560439,-0.203323,-0.564576,0.750364,0.700076,-0.345232,0.375900,0.068656,-0.193011,0.198889,0.020728,0.604746,-0.064802,-0.280133,-0.749602,-0.326104,-0.037335,0.231024,0.279959,0.301655,0.608201,-0.122703,-0.447051,0.020986,0.286003,0.346501,-0.823128,-0.264620,0.176721,-0.009595,-1.135337,0.485885,0.634706,0.816923,0.349658,0.906058,0.046738,-0.294949,0.204708,-0.141854,-0.804462,0.244340,-0.087588,-0.248015,0.201480,-0.450212,0.701273,0.727225,0.873402,0.254549,-0.133012,0.646283,0.404364,-0.097851,0.210423,-0.082445,-0.140194,-0.120666,0.369968,-0.072050,-0.615781,-1.122670,-0.874121,0.030103,-0.045933,-0.214983,0.339962,0.992562,0.041973,-0.188971,-1.009116,0.590533,-0.034519,0.321591,-0.265207,0.660194,-0.728218,0.613775,0.148525,-0.363351,0.389876,-0.055561,0.522080,0.482066,-0.531625,-0.749594,-0.156278,-0.172634,0.511477,-0.145456,-0.263025,-0.241127,-0.243496
1,0.657783,-1.139428,-1.409119,0.869928,-0.280212,-1.480376,-0.526623,-0.753044,-0.489739,0.118742,2.249381,-0.189061,0.563151,-1.089075,0.320481,1.378215,-1.269444,0.788732,-0.985927,0.179574,1.469556,1.115988,-0.261896,0.871474,-0.332202,0.135305,-1.254207,-0.047412,0.333694,-0.949805,-0.959544,-0.400107,0.641010,-1.987266,1.584039,-0.889744,1.687761,0.416291,1.726106,-0.551180,-1.270366,0.206664,0.143567,-0.237116,0.212676,-0.483928,-0.058783,0.322005,-1.034182,1.591441,0.558287,1.119671,0.441360,0.740791,0.531703,-0.476730,0.859004,0.184283,-1.698118,1.158969,0.334011,-0.155870,0.181241,-1.355326,0.944059,0.733871,0.605557,-0.132718,0.458136,-0.588934,0.200724,-1.084958,-1.833617,-1.529001,1.427796,-0.541753,0.115037,1.974111,-0.449561,0.409554,-0.237489,0.091197,-0.805269,0.827393,0.620039,-0.326176,0.302011,-1.412197,-0.118017,0.198602,-0.174066,-0.242182,-0.964918,-0.372071,0.370620,-0.022238,-1.142427,-2.186307,-0.685946,-1.134455,-0.383173,0.623782,-0.189035,-0.771120,1.075651,-0.106878,1.310569,0.745047,0.415386,-0.897564,1.911584,-0.547720,0.700422,-0.766923,-0.960457,0.648755,-0.661207,0.450936,-0.321271,-1.627941,-1.272138,0.215602,-1.274385,1.365814,0.952868,-1.032050,-0.966148,0.823783
2,-

In [248]:
emb_cate = df_business.apply(lambda row: cate_emb(row['categories']), axis='columns', result_type='expand')
emb_cate

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,-0.512302,0.530332,0.138834,-0.296921,-0.415271,-0.127761,-0.459205,-0.113282,-0.156458,-0.008257,1.072204,-0.279732,0.250593,-0.382305,0.012097,-0.148594,-0.107574,0.331740,-0.967119,-0.041754,0.159629,0.398388,-0.223930,-0.075650,-0.367091,-0.162557,0.042459,0.662939,-0.279451,-0.469043,-0.359846,0.278821,-0.623516,0.129731,-0.028047,0.333553,0.416740,0.779099,0.019547,-0.165711,-0.077700,0.728038,-0.421032,-0.314805,-0.116449,-0.440618,0.161705,0.479512,-0.017972,-0.062102,-0.072642,0.014889,0.579934,-0.034695,0.428723,0.570528,-0.652063,0.516356,-0.459616,0.119972,-0.369564,-0.305207,-0.058467,-0.326988,1.123963,-0.008338,-0.366706,0.114330,-0.422233,-0.364363,0.209441,-0.476591,0.098708,-1.108283,0.090141,-0.087093,0.165680,0.256206,-1.267640,0.295604,-0.170674,-0.253706,-0.405020,0.592657,0.238136,-0.035534,0.413829,-0.329256,-0.367246,0.395295,-0.849833,-0.216742,-0.384764,-0.305134,-0.116782,-0.124220,-0.388890,-0.419000,0.744997,-0.105861,-0.410814,0.965938,0.547050,0.205134,0.271868,-0.750555,0.257071,0.035046,-0.182045,-0.075403,0.625412,-0.430585,0.776703,0.043952,0.228010,-0.163617,0.465540,-0.390798,0.265744,0.062716,-0.123671,-0.707384,0.357379,0.024535,-0.634740,-0.076861,-0.455572,0.235973
1,-0.534540,-0.231857,0.113966,-0.361183,-0.119304,-0.247423,-0.392868,-0.155387,-0.192158,-0.056448,0.586422,-0.228426,-0.220382,-0.380589,-0.147784,-0.304246,0.227607,-0.257796,-0.316796,0.003430,0.400352,-0.179700,-0.064519,0.239999,-0.199747,0.168066,-0.115887,0.268115,-0.129371,-0.620698,-0.091733,0.078985,-0.216456,0.124697,0.435685,-0.040513,0.185485,0.540939,0.312610,-0.039736,-0.189362,0.885799,-0.347050,-0.251778,0.526280,-0.368705,0.179255,-0.371055,-0.280458,-0.139209,-0.034998,-0.033053,0.244704,0.186024,0.015127,0.325640,0.052719,0.030045,-0.362870,0.033742,-0.183841,-0.477037,0.174399,-0.011847,0.712471,0.435259,-0.176895,-0.049246,0.008360,-0.184535,0.196276,-0.437647,-0.167660,-0.485247,-0.189585,0.099429,-0.006437,0.276313,-0.623958,0.962294,-0.348562,0.257395,-0.152183,0.174228,-0.141833,-0.166660,0.125948,-0.162841,0.013326,0.162888,-0.399778,-0.287691,0.254276,-0.483528,0.327016,-0.217572,-0.371276,-0.056853,0.033454,-0.084465,-0.089488,0.293804,-0.067374,0.190793,0.323900,-0.342455,-0.019002,-0.302270,0.165578,-0.072799,-0.021501,-0.153341,0.812292,-0.074961,-0.124646,0.360822,0.110268,-0.055843,-0.034522,0.114849,-0.393591,-0.030674,-0.322591,0.217973,0.455055,-0.194363,-0.371525,0.009935
2,-0.132113,0.428914,0.083771,0.006295,0.336714,-0.253058,-0.290321,0.043733,-0.238493,0.217002,0.558310,-0.051575,-0.046221,-0.400791,0.129725,-0.138744,0.158402,0.159172,-0.123597,0.336208,0.288642,-0.356445,-0.231220,0.074476,-0.204654,-0.028128,-0.329637,0.193056,-0.060430,-0.689258,-0.070101,-0.056582,0.046561,0.092237,0.315992,-0.081166,0.569140,0.354352,0.400627,0.230894,-0.372566,0.668679,-0.099402,-0.317079,0.274700,-0.248846,0.192875,-0.023321,-0.162917,-0.283321,0.072338,0.223350,0.585238,-0.158337,0.006048,0.707916,0.128513,0.363356,-0.362406,-0.032643,0.281848,0.015997,0.073955,-0.000816,0.701690,0.158589,-0.074763,0.128724,-0.234216,-0.206991,0.159819,-0.172107,0.004872,-0.077941,-0.091270,-0.439453,0.159840,0.291371,-0.425071,0.879834,-0.240207,-0.178712,-0.342053,0.405548,-0.201877,0.279141,0.138068,0.005131,0.355276,0.051153,-0.258919,-0.326186,-0.083584,-0.545733,0.161400,-0.278834,-0.435659,-0.251933,-0.117866,0.036034,-0.366075,0.571895,-0.062026,0.282366,0.225212,-0.370689,0.038980,-0.216061,0.141354,0.179109,0.344016,-0.160752,0.662162,-0.206845,-0.205297,-0.075391,0.194450,-0.130025,-0.395261,-0.02

In [249]:
emb_cate_name = {}
for i in range(128):
    emb_cate_name[i] = 'emb_cate_' + str(i)
emb_cate_name

{0: 'emb_cate_0',
 1: 'emb_cate_1',
 2: 'emb_cate_2',
 3: 'emb_cate_3',
 4: 'emb_cate_4',
 5: 'emb_cate_5',
 6: 'emb_cate_6',
 7: 'emb_cate_7',
 8: 'emb_cate_8',
 9: 'emb_cate_9',
 10: 'emb_cate_10',
 11: 'emb_cate_11',
 12: 'emb_cate_12',
 13: 'emb_cate_13',
 14: 'emb_cate_14',
 15: 'emb_cate_15',
 16: 'emb_cate_16',
 17: 'emb_cate_17',
 18: 'emb_cate_18',
 19: 'emb_cate_19',
 20: 'emb_cate_20',
 21: 'emb_cate_21',
 22: 'emb_cate_22',
 23: 'emb_cate_23',
 24: 'emb_cate_24',
 25: 'emb_cate_25',
 26: 'emb_cate_26',
 27: 'emb_cate_27',
 28: 'emb_cate_28',
 29: 'emb_cate_29',
 30: 'emb_cate_30',
 31: 'emb_cate_31',
 32: 'emb_cate_32',
 33: 'emb_cate_33',
 34: 'emb_cate_34',
 35: 'emb_cate_35',
 36: 'emb_cate_36',
 37: 'emb_cate_37',
 38: 'emb_cate_38',
 39: 'emb_cate_39',
 40: 'emb_cate_40',
 41: 'emb_cate_41',
 42: 'emb_cate_42',
 43: 'emb_cate_43',
 44: 'emb_cate_44',
 45: 'emb_cate_45',
 46: 'emb_cate_46',
 47: 'emb_cate_47',
 48: 'emb_cate_48',
 49: 'emb_cate_49',
 50: 'emb_cate_50',


In [250]:
emb_cate = emb_cate.rename(columns=emb_cate_name)
emb_cate

,emb_cate_0,emb_cate_1,emb_cate_2,emb_cate_3,emb_cate_4,emb_cate_5,emb_cate_6,emb_cate_7,emb_cate_8,emb_cate_9,emb_cate_10,emb_cate_11,emb_cate_12,emb_cate_13,emb_cate_14,emb_cate_15,emb_cate_16,emb_cate_17,emb_cate_18,emb_cate_19,emb_cate_20,emb_cate_21,emb_cate_22,emb_cate_23,emb_cate_24,emb_cate_25,emb_cate_26,emb_cate_27,emb_cate_28,emb_cate_29,emb_cate_30,emb_cate_31,emb_cate_32,emb_cate_33,emb_cate_34,emb_cate_35,emb_cate_36,emb_cate_37,emb_cate_38,emb_cate_39,emb_cate_40,emb_cate_41,emb_cate_42,emb_cate_43,emb_cate_44,emb_cate_45,emb_cate_46,emb_cate_47,emb_cate_48,emb_cate_49,emb_cate_50,emb_cate_51,emb_cate_52,emb_cate_53,emb_cate_54,emb_cate_55,emb_cate_56,emb_cate_57,emb_cate_58,emb_cate_59,emb_cate_60,emb_cate_61,emb_cate_62,emb_cate_63,emb_cate_64,emb_cate_65,emb_cate_66,emb_cate_67,emb_cate_68,emb_cate_69,emb_cate_70,emb_cate_71,emb_cate_72,emb_cate_73,emb_cate_74,emb_cate_75,emb_cate_76,emb_cate_77,emb_cate_78,emb_cate_79,emb_cate_80,emb_cate_81,emb_cate_82,emb_cate_83,emb_cate_84,emb_cate_85,emb_cate_86,emb_cate_87,emb_cate_88,emb_cate_89,emb_cate_90,emb_cate_91,emb_cate_92,emb_cate_93,emb_cate_94,emb_cate_95,emb_cate_96,emb_cate_97,emb_cate_98,emb_cate_99,emb_cate_100,emb_cate_101,emb_cate_102,emb_cate_103,emb_cate_104,emb_cate_105,emb_cate_106,emb_cate_107,emb_cate_108,emb_cate_109,emb_cate_110,emb_cate_111,emb_cate_112,emb_cate_113,emb_cate_114,emb_cate_115,emb_cate_116,emb_cate_117,emb_cate_118,emb_cate_119,emb_cate_120,emb_cate_121,emb_cate_122,emb_cate_123,emb_cate_124,emb_cate_125,emb_cate_126,emb_cate_127
0,-0.512302,0.530332,0.138834,-0.296921,-0.415271,-0.127761,-0.459205,-0.113282,-0.156458,-0.008257,1.072204,-0.279732,0.250593,-0.382305,0.012097,-0.148594,-0.107574,0.331740,-0.967119,-0.041754,0.159629,0.398388,-0.223930,-0.075650,-0.367091,-0.162557,0.042459,0.662939,-0.279451,-0.469043,-0.359846,0.278821,-0.623516,0.129731,-0.028047,0.333553,0.416740,0.779099,0.019547,-0.165711,-0.077700,0.728038,-0.421032,-0.314805,-0.116449,-0.440618,0.161705,0.479512,-0.017972,-0.062102,-0.072642,0.014889,0.579934,-0.034695,0.428723,0.570528,-0.652063,0.516356,-0.459616,0.119972,-0.369564,-0.305207,-0.058467,-0.326988,1.123963,-0.008338,-0.366706,0.114330,-0.422233,-0.364363,0.209441,-0.476591,0.098708,-1.108283,0.090141,-0.087093,0.165680,0.256206,-1.267640,0.295604,-0.170674,-0.253706,-0.405020,0.592657,0.238136,-0.035534,0.413829,-0.329256,-0.367246,0.395295,-0.849833,-0.216742,-0.384764,-0.305134,-0.116782,-0.124220,-0.388890,-0.419000,0.744997,-0.105861,-0.410814,0.965938,0.547050,0.205134,0.271868,-0.750555,0.257071,0.035046,-0.182045,-0.075403,0.625412,-0.430585,0.776703,0.043952,0.228010,-0.163617,0.465540,-0.390798,0.265744,0.062716,-0.123671,-0.707384,0.357379,0.024535,-0.634740,-0.076861,-0.455572,0.235973
1,-0.534540,-0.231857,0.113966,-0.361183,-0.119304,-0.247423,-0.392868,-0.155387,-0.192158,-0.056448,0.586422,-0.228426,-0.220382,-0.380589,-0.147784,-0.304246,0.227607,-0.257796,-0.316796,0.003430,0.400352,-0.179700,-0.064519,0.239999,-0.199747,0.168066,-0.115887,0.268115,-0.129371,-0.620698,-0.091733,0.078985,-0.216456,0.124697,0.435685,-0.040513,0.185485,0.540939,0.312610,-0.039736,-0.189362,0.885799,-0.347050,-0.251778,0.526280,-0.368705,0.179255,-0.371055,-0.280458,-0.139209,-0.034998,-0.033053,0.244704,0.186024,0.015127,0.325640,0.052719,0.030045,-0.362870,0.033742,-0.183841,-0.477037,0.174399,-0.011847,0.712471,0.435259,-0.176895,-0.049246,0.008360,-0.184535,0.196276,-0.437647,-0.167660,-0.485247,-0.189585,0.099429,-0.006437,0.276313,-0.623958,0.962294,-0.348562,0.257395,-0.152183,0.174228,-0.141833,-0.166660,0.125948,-0.162841,0.013326,0.162888,-0.399778,-0.287691,0.254276,-0.483528,0.327016,-0.217572,-0.371276,-0.056853,0.033454,-0.084465,-0.089488,0.293804,-0.067374,0.190793,0.323900,-0.342455,-0.019002,-0.302270,0.165578,-0.072799,-0.021501,-0.153341,0.812292,-0.074961,-0.124646,0.360822,0.110268,-0.055843,-0.034522,0.114849,-0.393591,-0.030674,-0.322591,0.217973,0.455055,-0.194363,-0.37

In [251]:
df_business.drop(columns=['categories','city'], inplace=True)
df_business = pd.concat([df_business, emb_city, emb_cate], axis='columns').fillna(0)
df_business

,business_id,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb_city_81,emb_city_82,emb_city_83,emb_city_84,emb_city_85,emb_city_86,emb_city_87,emb_city_88,emb_city_89,emb_city_90,emb_city_91,emb_city_92,emb_city_93,emb_city_94,emb_city_95,emb_city_96,emb_city_97,emb_city_98,emb_city_99,emb_city_100,emb_city_101,emb_city_102,emb_city_103,emb_city_104,emb_city_105,emb_city_106,emb_city_107,emb_city_108,emb_city_109,emb_city_110,emb_city_111,emb_city_112,emb_city_113,emb_city_114,emb_city_115,emb_city_116,emb_city_117,emb_city_118,emb_city_119,emb_city_120,emb_city_121,emb_city_122,emb_city_123,emb_city_124,emb_city_125,emb_city_126,emb_city_127,emb_cate_0,emb_cate_1,emb_cate_2,emb_cate_3,emb_cate_4,emb_cate_5,emb_cate_6,emb_cate_7,emb_cate_8,emb_cate_9,emb_cate_10,emb_cate_11,emb_cate_12,emb_cate_13,emb_cate_14,emb_cate_15,emb_cate_16,emb_cate_17,emb_cate_18,emb_cate_19,emb_cate_20,emb_cate_21,emb_cate_22,emb_cate_23,emb_cate_24,emb_cate_25,emb_cate_26,emb_cate_27,emb_cate_28,emb_cate_29,emb_cate_30,emb_cate_31,emb_cate_32,emb_cate_33,emb_cate_34,emb_cate_35,emb_cate_36,emb_cate_37,emb_cate_38,emb_cate_39,emb_cate_40,emb_cate_41,emb_cate_42,emb_cate_43,emb_cate_44,emb_cate_45,emb_cate_46,emb_cate_47,emb_cate_48,emb_cate_49,emb_cate_50,emb_cate_51,emb_cate_52,emb_cate_53,emb_cate_54,emb_cate_55,emb_cate_56,emb_cate_57,emb_cate_58,emb_cate_59,emb_cate_60,emb_cate_61,emb_cate_62,emb_cate_63,emb_cate_64,emb_cate_65,emb_cate_66,emb_cate_67,emb_cate_68,emb_cate_69,emb_cate_70,emb_cate_71,emb_cate_72,emb_cate_73,emb_cate_74,emb_cate_75,emb_cate_76,emb_cate_77,emb_cate_78,emb_cate_79,emb_cate_80,emb_cate_81,emb_cate_82,emb_cate_83,emb_cate_84,emb_cate_85,emb_cate_86,emb_cate_87,emb_cate_88,emb_cate_89,emb_cate_90,emb_cate_91,emb_cate_92,emb_cate_93,emb_cate_94,emb_cate_95,emb_cate_96,emb_cate_97,emb_cate_98,emb_cate_99,emb_cate_100,emb_cate_101,emb_cate_102,emb_cate_103,emb_cate_104,emb_cate_105,emb_cate_106,emb_cate_107,emb_cate_108,emb_cate_109,emb_cate_110,emb_cate_111,emb_cate_112,emb_cate_113,emb_cate_114,emb_cate_115,emb_cate_116,emb_cate_117,emb_cate_118,emb_cate_119,emb_cate_120,emb_cate_121,emb_cate_122,emb_cate_123,emb_cate_124,emb_cate_125,emb_cate_126,emb_cate_127
0,tZnSodhPwNr4bzrwJ1CSbw,6,0,4.0,146,3608,0.019304,-0.177057,0.637373,0.318220,-0.006698,-0.545345,-0.306781,-0.125858,-0.793541,-0.208935,0.773403,-0.016629,0.304182,-0.245043,-0.328879,-0.640849,0.278763,0.021803,-0.735101,0.685369,0.545151,-0.221520,0.033404,0.220473,-0.236153,-0.016998,0.250888,0.879612,-0.826165,-0.138453,-0.347213,0.421021,-0.505975,-0.160681,0.236192,-0.560439,-0.203323,-0.564576,0.750364,0.700076,-0.345232,0.375900,0.068656,-0.193011,0.198889,0.020728,0.604746,-0.064802,-0.280133,-0.749602,-0.326104,-0.037335,0.231024,0.279959,0.301655,0.608201,-0.122703,-0.447051,0.020986,0.286003,0.346501,-0.823128,-0.264620,0.176721,-0.009595,-1.135337,0.485885,0.634706,0.816923,0.349658,0.906058,0.046738,-0.294949,0.204708,-0.141854,-0.804462,0.244340,-0.087588,-0.248015,0.201480,-0.450212,0.701273,0.727225,0

In [252]:
df_business.to_csv('/data/chaoyu/data553/business.csv', index=False)

In [180]:
business_features = pd.read_csv('/data/chaoyu/data553/business_train.csv')

In [181]:
business_features

,business_id,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb_city_81,emb_city_82,emb_city_83,emb_city_84,emb_city_85,emb_city_86,emb_city_87,emb_city_88,emb_city_89,emb_city_90,emb_city_91,emb_city_92,emb_city_93,emb_city_94,emb_city_95,emb_city_96,emb_city_97,emb_city_98,emb_city_99,emb_city_100,emb_city_101,emb_city_102,emb_city_103,emb_city_104,emb_city_105,emb_city_106,emb_city_107,emb_city_108,emb_city_109,emb_city_110,emb_city_111,emb_city_112,emb_city_113,emb_city_114,emb_city_115,emb_city_116,emb_city_117,emb_city_118,emb_city_119,emb_city_120,emb_city_121,emb_city_122,emb_city_123,emb_city_124,emb_city_125,emb_city_126,emb_city_127,emb_cate_0,emb_cate_1,emb_cate_2,emb_cate_3,emb_cate_4,emb_cate_5,emb_cate_6,emb_cate_7,emb_cate_8,emb_cate_9,emb_cate_10,emb_cate_11,emb_cate_12,emb_cate_13,emb_cate_14,emb_cate_15,emb_cate_16,emb_cate_17,emb_cate_18,emb_cate_19,emb_cate_20,emb_cate_21,emb_cate_22,emb_cate_23,emb_cate_24,emb_cate_25,emb_cate_26,emb_cate_27,emb_cate_28,emb_cate_29,emb_cate_30,emb_cate_31,emb_cate_32,emb_cate_33,emb_cate_34,emb_cate_35,emb_cate_36,emb_cate_37,emb_cate_38,emb_cate_39,emb_cate_40,emb_cate_41,emb_cate_42,emb_cate_43,emb_cate_44,emb_cate_45,emb_cate_46,emb_cate_47,emb_cate_48,emb_cate_49,emb_cate_50,emb_cate_51,emb_cate_52,emb_cate_53,emb_cate_54,emb_cate_55,emb_cate_56,emb_cate_57,emb_cate_58,emb_cate_59,emb_cate_60,emb_cate_61,emb_cate_62,emb_cate_63,emb_cate_64,emb_cate_65,emb_cate_66,emb_cate_67,emb_cate_68,emb_cate_69,emb_cate_70,emb_cate_71,emb_cate_72,emb_cate_73,emb_cate_74,emb_cate_75,emb_cate_76,emb_cate_77,emb_cate_78,emb_cate_79,emb_cate_80,emb_cate_81,emb_cate_82,emb_cate_83,emb_cate_84,emb_cate_85,emb_cate_86,emb_cate_87,emb_cate_88,emb_cate_89,emb_cate_90,emb_cate_91,emb_cate_92,emb_cate_93,emb_cate_94,emb_cate_95,emb_cate_96,emb_cate_97,emb_cate_98,emb_cate_99,emb_cate_100,emb_cate_101,emb_cate_102,emb_cate_103,emb_cate_104,emb_cate_105,emb_cate_106,emb_cate_107,emb_cate_108,emb_cate_109,emb_cate_110,emb_cate_111,emb_cate_112,emb_cate_113,emb_cate_114,emb_cate_115,emb_cate_116,emb_cate_117,emb_cate_118,emb_cate_119,emb_cate_120,emb_cate_121,emb_cate_122,emb_cate_123,emb_cate_124,emb_cate_125,emb_cate_126,emb_cate_127
0,tZnSodhPwNr4bzrwJ1CSbw,5,0,4.0,146,3598,-0.261519,-0.599941,0.207167,1.081511,0.143446,-0.990259,-0.161810,0.314183,-0.562352,-1.032633,0.468810,-0.138627,-0.458146,-0.200151,-0.775774,0.154883,0.019745,-0.237248,-0.326067,0.012721,0.064079,0.507143,-0.083561,0.106925,0.283722,-0.172801,-0.237359,0.240721,-0.616283,-0.468782,-1.061548,-0.313204,-0.715688,-0.055313,-0.057600,0.284490,0.594202,-0.380060,0.020300,0.230236,-0.376869,-0.130621,-0.103166,0.257418,0.075174,0.935234,0.102692,0.369581,-0.553402,-0.606574,0.280283,-1.007279,0.438781,0.170910,-0.369890,0.696780,0.125976,-0.549884,-0.350193,0.579836,0.114678,-0.919959,0.204027,0.103701,-0.138248,-0.780091,-0.313870,-0.103997,0.047241,0.019522,-0.074081,0.513874,-0.219033,-0.545058,0.217559,-0.465467,-0.051494,-0.718019,-0.358849,0.397164,-0.389036,0.276109,1.10103

In [255]:
df_user = pd.read_json('/data/chaoyu/data553/user.json', lines=True)
df_user

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,lzlZwIpuSWXEnNS91wxjHw,Susan,1,2015-09-28,None,0,0,0,0,None,2.00,0,0,0,0,0,0,0,0,0,0,0
1,XvLBr-9smbI0m_a7dXtB7w,Daipayan,2,2015-09-05,None,0,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,0
2,QPT4Ud4H5sJVr68yXhoWFw,Andy,1,2016-07-21,None,0,0,0,0,None,4.00,0,0,0,0,0,0,0,0,0,0,0
3,i5YitlHZpf0B3R0s_8NVuw,Jonathan,19,2014-08-04,None,0,0,0,0,None,4.05,0,0,0,0,0,0,0,0,0,0,0
4,s4FoIXE_LSGviTHBe8dmcg,Shashank,3,2017-06-18,None,0,0,0,0,None,3.00,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518164,AwyHq4AtHU-Ds1khPUIIpQ,Stephanie,1,2018-03-02,None,0,0,0,0,None,4.00,0,0,0,0,0,0,0,0,0,0,0
1518165,dZ-qwZX1_Ge-8KuSIJHhyA,Nicholas,4,2015-12-09,None,0,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,0
1518166,Mv2-NyScx6SSvu2Swc5BEQ,Danielle,1,2014-12-20,None,0,0,0,1,None,1.00,0,0,0,0,0,0,0,0,0,0,0
1518167,9STgncH6CB8bXdcp2OjIuA,Eden,14,2015-02-21,None,0,0,0,0,None,3.73,0,0,0,0,0,0,0,0,0,0,0


In [256]:
df_user['review_count_u'] = df_user['review_count']
df_user['yelping_since'] = df_user['yelping_since'].map(lambda row: row[:4])
df_user['yelping_since'] = lbe.fit_transform(df_user['yelping_since'])
df_user

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u
0,lzlZwIpuSWXEnNS91wxjHw,Susan,1,11,None,0,0,0,0,None,2.00,0,0,0,0,0,0,0,0,0,0,0,1
1,XvLBr-9smbI0m_a7dXtB7w,Daipayan,2,11,None,0,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,0,2
2,QPT4Ud4H5sJVr68yXhoWFw,Andy,1,12,None,0,0,0,0,None,4.00,0,0,0,0,0,0,0,0,0,0,0,1
3,i5YitlHZpf0B3R0s_8NVuw,Jonathan,19,10,None,0,0,0,0,None,4.05,0,0,0,0,0,0,0,0,0,0,0,19
4,s4FoIXE_LSGviTHBe8dmcg,Shashank,3,13,None,0,0,0,0,None,3.00,0,0,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518164,AwyHq4AtHU-Ds1khPUIIpQ,Stephanie,1,14,None,0,0,0,0,None,4.00,0,0,0,0,0,0,0,0,0,0,0,1
1518165,dZ-qwZX1_Ge-8KuSIJHhyA,Nicholas,4,11,None,0,0,0,0,None,5.00,0,0,0,0,0,0,0,0,0,0,0,4
1518166,Mv2-NyScx6SSvu2Swc5BEQ,Danielle,1,10,None,0,0,0,1,None,1.00,0,0,0,0,0,0,0,0,0,0,0,1
1518167,9STgncH6CB8bXdcp2OjIuA,Eden,14,11,None,0,0,0,0,None,3.73,0,0,0,0,0,0,0,0,0,0,0,14


In [257]:
user_features = df_user.drop(['name', 'friends', 'elite', 'review_count'], axis=1)
user_features

,user_id,yelping_since,useful,funny,cool,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_count_u
0,lzlZwIpuSWXEnNS91wxjHw,11,0,0,0,0,2.00,0,0,0,0,0,0,0,0,0,0,0,1
1,XvLBr-9smbI0m_a7dXtB7w,11,0,0,0,0,5.00,0,0,0,0,0,0,0,0,0,0,0,2
2,QPT4Ud4H5sJVr68yXhoWFw,12,0,0,0,0,4.00,0,0,0,0,0,0,0,0,0,0,0,1
3,i5YitlHZpf0B3R0s_8NVuw,10,0,0,0,0,4.05,0,0,0,0,0,0,0,0,0,0,0,19
4,s4FoIXE_LSGviTHBe8dmcg,13,0,0,0,0,3.00,0,0,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518164,AwyHq4AtHU-Ds1khPUIIpQ,14,0,0,0,0,4.00,0,0,0,0,0,0,0,0,0,0,0,1
1518165,dZ-qwZX1_Ge-8KuSIJHhyA,11,0,0,0,0,5.00,0,0,0,0,0,0,0,0,0,0,0,4
1518166,Mv2-NyScx6SSvu2Swc5BEQ,10,0,0,0,1,1.00,0,0,0,0,0,0,0,0,0,0,0,1
1518167,9STgncH6CB8bXdcp2OjIuA,11,0,0,0,0,3.73,0,0,0,0,0,0,0,0,0,0,0,14


In [258]:
user_features.to_csv('/data/chaoyu/data553/user.csv', index=False)

In [182]:
user_features = pd.read_csv('/data/chaoyu/data553/user_train.csv')

In [199]:
#Train_only
train_data = df_train.merge(embeddings, left_on='business_id', right_on='id', how='left')
train_data = train_data.merge(embeddings, left_on='user_id', right_on='id', how='left', suffixes=('_bus', '_user'))
train_data = train_data.merge(business_features, on='business_id', how='left')
train_data = train_data.merge(user_features, on='user_id', how='left')
train_data.drop(['user_id', 'business_id', 'id_bus', 'id_user'], axis=1, inplace=True)

#User
#train_data = df_train.merge(embeddings, left_on='user_id', right_on='id', how='left')
#train_data = train_data.merge(business_features, on='business_id', how='left')
#train_data = train_data.merge(user_features, on='user_id', how='left')
#train_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)

#Business
#train_data = df_train.merge(embeddings, left_on='business_id', right_on='id', how='left')
#train_data = train_data.merge(business_features, on='business_id', how='left')
#train_data = train_data.merge(user_features, on='user_id', how='left')
#train_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)

#Null
#train_data = df_train.merge(business_features, on='business_id', how='left')
#train_data = train_data.merge(user_features, on='user_id', how='left')
#train_data.drop(['user_id', 'business_id'], axis=1, inplace=True)

#Combiantion
#train_data = df_comb.merge(embeddings, left_on='business_id', right_on='id', how='left')
#train_data = train_data.merge(embeddings, left_on='user_id', right_on='id', how='left', suffixes=('_bus', '_user'))
#train_data = train_data.merge(business_features, on='business_id', how='left')
#train_data = train_data.merge(user_features, on='user_id', how='left')
#train_data.drop(['user_id', 'business_id', 'id_bus', 'id_user'], axis=1, inplace=True)

#Combination User
#train_data = df_comb.merge(embeddings, left_on='user_id', right_on='id', how='left')
#train_data = train_data.merge(business_features, on='business_id', how='left')
#train_data = train_data.merge(user_features, on='user_id', how='left')
#train_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)

#Combination Business
#train_data = df_comb.merge(embeddings, left_on='business_id', right_on='id', how='left')
#train_data = train_data.merge(business_features, on='business_id', how='left')
#train_data = train_data.merge(user_features, on='user_id', how='left')
#train_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)

#Combination Null
#train_data = df_comb.merge(business_features, on='business_id', how='left')
#train_data = train_data.merge(user_features, on='user_id', how='left')
#train_data.drop(['user_id', 'business_id'], axis=1, inplace=True)
train_data

,target,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,emb_10_bus,emb_11_bus,emb_12_bus,emb_13_bus,emb_14_bus,emb_15_bus,emb_16_bus,emb_17_bus,emb_18_bus,emb_19_bus,emb_20_bus,emb_21_bus,emb_22_bus,emb_23_bus,emb_24_bus,emb_25_bus,emb_26_bus,emb_27_bus,emb_28_bus,emb_29_bus,emb_30_bus,emb_31_bus,emb_32_bus,emb_33_bus,emb_34_bus,emb_35_bus,emb_36_bus,emb_37_bus,emb_38_bus,emb_39_bus,emb_40_bus,emb_41_bus,emb_42_bus,emb_43_bus,emb_44_bus,emb_45_bus,emb_46_bus,emb_47_bus,emb_48_bus,emb_49_bus,emb_50_bus,emb_51_bus,emb_52_bus,emb_53_bus,emb_54_bus,emb_55_bus,emb_56_bus,emb_57_bus,emb_58_bus,emb_59_bus,emb_60_bus,emb_61_bus,emb_62_bus,emb_63_bus,emb_64_bus,emb_65_bus,emb_66_bus,emb_67_bus,emb_68_bus,emb_69_bus,emb_70_bus,emb_71_bus,emb_72_bus,emb_73_bus,emb_74_bus,emb_75_bus,emb_76_bus,emb_77_bus,emb_78_bus,emb_79_bus,emb_80_bus,emb_81_bus,emb_82_bus,emb_83_bus,emb_84_bus,emb_85_bus,emb_86_bus,emb_87_bus,emb_88_bus,emb_89_bus,emb_90_bus,emb_91_bus,emb_92_bus,emb_93_bus,emb_94_bus,emb_95_bus,emb_96_bus,emb_97_bus,emb_98_bus,emb_99_bus,emb_100_bus,emb_101_bus,emb_102_bus,emb_103_bus,emb_104_bus,emb_105_bus,emb_106_bus,emb_107_bus,emb_108_bus,emb_109_bus,emb_110_bus,emb_111_bus,emb_112_bus,emb_113_bus,emb_114_bus,emb_115_bus,emb_116_bus,emb_117_bus,emb_118_bus,emb_119_bus,emb_120_bus,emb_121_bus,emb_122_bus,emb_123_bus,emb_124_bus,emb_125_bus,emb_126_bus,emb_127_bus,emb_0_user,emb_1_user,emb_2_user,emb_3_user,emb_4_user,emb_5_user,emb_6_user,emb_7_user,emb_8_user,emb_9_user,emb_10_user,emb_11_user,emb_12_user,emb_13_user,emb_14_user,emb_15_user,emb_16_user,emb_17_user,emb_18_user,emb_19_user,emb_20_user,emb_21_user,emb_22_user,emb_23_user,emb_24_user,emb_25_user,emb_26_user,emb_27_user,emb_28_user,emb_29_user,emb_30_user,emb_31_user,emb_32_user,emb_33_user,emb_34_user,emb_35_user,emb_36_user,emb_37_user,emb_38_user,emb_39_user,emb_40_user,emb_41_user,emb_42_user,emb_43_user,emb_44_user,emb_45_user,emb_46_user,emb_47_user,emb_48_user,emb_49_user,emb_50_user,emb_51_user,emb_52_user,emb_53_user,emb_54_user,emb_55_user,emb_56_user,emb_57_user,emb_58_user,emb_59_user,emb_60_user,emb_61_user,emb_62_user,emb_63_user,emb_64_user,emb_65_user,emb_66_user,emb_67_user,emb_68_user,emb_69_user,emb_70_user,emb_71_user,emb_72_user,emb_73_user,emb_74_user,emb_75_user,emb_76_user,emb_77_user,emb_78_user,emb_79_user,emb_80_user,emb_81_user,emb_82_user,emb_83_user,emb_84_user,emb_85_user,emb_86_user,emb_87_user,emb_88_user,emb_89_user,emb_90_user,emb_91_user,emb_92_user,emb_93_user,emb_94_user,emb_95_user,emb_96_user,emb_97_user,emb_98_user,emb_99_user,emb_100_user,emb_101_user,emb_102_user,emb_103_user,emb_104_user,emb_105_user,emb_106_user,emb_107_user,emb_108_user,emb_109_user,emb_110_user,emb_111_user,emb_112_user,emb_113_user,emb_114_user,emb_115_user,emb_116_user,emb_117_user,emb_118_user,emb_119_user,emb_120_user,emb_121_user,emb_122_user,emb_123_user,emb_124_user,emb_125_user,emb_126_user,emb_127_user,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb

In [200]:
print(train_data.columns.values.tolist())

['target', 'emb_0_bus', 'emb_1_bus', 'emb_2_bus', 'emb_3_bus', 'emb_4_bus', 'emb_5_bus', 'emb_6_bus', 'emb_7_bus', 'emb_8_bus', 'emb_9_bus', 'emb_10_bus', 'emb_11_bus', 'emb_12_bus', 'emb_13_bus', 'emb_14_bus', 'emb_15_bus', 'emb_16_bus', 'emb_17_bus', 'emb_18_bus', 'emb_19_bus', 'emb_20_bus', 'emb_21_bus', 'emb_22_bus', 'emb_23_bus', 'emb_24_bus', 'emb_25_bus', 'emb_26_bus', 'emb_27_bus', 'emb_28_bus', 'emb_29_bus', 'emb_30_bus', 'emb_31_bus', 'emb_32_bus', 'emb_33_bus', 'emb_34_bus', 'emb_35_bus', 'emb_36_bus', 'emb_37_bus', 'emb_38_bus', 'emb_39_bus', 'emb_40_bus', 'emb_41_bus', 'emb_42_bus', 'emb_43_bus', 'emb_44_bus', 'emb_45_bus', 'emb_46_bus', 'emb_47_bus', 'emb_48_bus', 'emb_49_bus', 'emb_50_bus', 'emb_51_bus', 'emb_52_bus', 'emb_53_bus', 'emb_54_bus', 'emb_55_bus', 'emb_56_bus', 'emb_57_bus', 'emb_58_bus', 'emb_59_bus', 'emb_60_bus', 'emb_61_bus', 'emb_62_bus', 'emb_63_bus', 'emb_64_bus', 'emb_65_bus', 'emb_66_bus', 'emb_67_bus', 'emb_68_bus', 'emb_69_bus', 'emb_70_bus', 'emb_

In [201]:
pd.set_option('display.max_columns', None)
train_data

,target,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,emb_10_bus,emb_11_bus,emb_12_bus,emb_13_bus,emb_14_bus,emb_15_bus,emb_16_bus,emb_17_bus,emb_18_bus,emb_19_bus,emb_20_bus,emb_21_bus,emb_22_bus,emb_23_bus,emb_24_bus,emb_25_bus,emb_26_bus,emb_27_bus,emb_28_bus,emb_29_bus,emb_30_bus,emb_31_bus,emb_32_bus,emb_33_bus,emb_34_bus,emb_35_bus,emb_36_bus,emb_37_bus,emb_38_bus,emb_39_bus,emb_40_bus,emb_41_bus,emb_42_bus,emb_43_bus,emb_44_bus,emb_45_bus,emb_46_bus,emb_47_bus,emb_48_bus,emb_49_bus,emb_50_bus,emb_51_bus,emb_52_bus,emb_53_bus,emb_54_bus,emb_55_bus,emb_56_bus,emb_57_bus,emb_58_bus,emb_59_bus,emb_60_bus,emb_61_bus,emb_62_bus,emb_63_bus,emb_64_bus,emb_65_bus,emb_66_bus,emb_67_bus,emb_68_bus,emb_69_bus,emb_70_bus,emb_71_bus,emb_72_bus,emb_73_bus,emb_74_bus,emb_75_bus,emb_76_bus,emb_77_bus,emb_78_bus,emb_79_bus,emb_80_bus,emb_81_bus,emb_82_bus,emb_83_bus,emb_84_bus,emb_85_bus,emb_86_bus,emb_87_bus,emb_88_bus,emb_89_bus,emb_90_bus,emb_91_bus,emb_92_bus,emb_93_bus,emb_94_bus,emb_95_bus,emb_96_bus,emb_97_bus,emb_98_bus,emb_99_bus,emb_100_bus,emb_101_bus,emb_102_bus,emb_103_bus,emb_104_bus,emb_105_bus,emb_106_bus,emb_107_bus,emb_108_bus,emb_109_bus,emb_110_bus,emb_111_bus,emb_112_bus,emb_113_bus,emb_114_bus,emb_115_bus,emb_116_bus,emb_117_bus,emb_118_bus,emb_119_bus,emb_120_bus,emb_121_bus,emb_122_bus,emb_123_bus,emb_124_bus,emb_125_bus,emb_126_bus,emb_127_bus,emb_0_user,emb_1_user,emb_2_user,emb_3_user,emb_4_user,emb_5_user,emb_6_user,emb_7_user,emb_8_user,emb_9_user,emb_10_user,emb_11_user,emb_12_user,emb_13_user,emb_14_user,emb_15_user,emb_16_user,emb_17_user,emb_18_user,emb_19_user,emb_20_user,emb_21_user,emb_22_user,emb_23_user,emb_24_user,emb_25_user,emb_26_user,emb_27_user,emb_28_user,emb_29_user,emb_30_user,emb_31_user,emb_32_user,emb_33_user,emb_34_user,emb_35_user,emb_36_user,emb_37_user,emb_38_user,emb_39_user,emb_40_user,emb_41_user,emb_42_user,emb_43_user,emb_44_user,emb_45_user,emb_46_user,emb_47_user,emb_48_user,emb_49_user,emb_50_user,emb_51_user,emb_52_user,emb_53_user,emb_54_user,emb_55_user,emb_56_user,emb_57_user,emb_58_user,emb_59_user,emb_60_user,emb_61_user,emb_62_user,emb_63_user,emb_64_user,emb_65_user,emb_66_user,emb_67_user,emb_68_user,emb_69_user,emb_70_user,emb_71_user,emb_72_user,emb_73_user,emb_74_user,emb_75_user,emb_76_user,emb_77_user,emb_78_user,emb_79_user,emb_80_user,emb_81_user,emb_82_user,emb_83_user,emb_84_user,emb_85_user,emb_86_user,emb_87_user,emb_88_user,emb_89_user,emb_90_user,emb_91_user,emb_92_user,emb_93_user,emb_94_user,emb_95_user,emb_96_user,emb_97_user,emb_98_user,emb_99_user,emb_100_user,emb_101_user,emb_102_user,emb_103_user,emb_104_user,emb_105_user,emb_106_user,emb_107_user,emb_108_user,emb_109_user,emb_110_user,emb_111_user,emb_112_user,emb_113_user,emb_114_user,emb_115_user,emb_116_user,emb_117_user,emb_118_user,emb_119_user,emb_120_user,emb_121_user,emb_122_user,emb_123_user,emb_124_user,emb_125_user,emb_126_user,emb_127_user,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb

In [202]:
train_dense = ['stars', 'review_count_b', 'review_count_u', 'useful', 'funny', 'cool', 'fans', 
               'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 
               'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 
               'compliment_writer', 'compliment_photos']
mms = MinMaxScaler(feature_range=(0,1))
train_data[train_dense] = mms.fit_transform(train_data[train_dense])
train_data

,target,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,emb_10_bus,emb_11_bus,emb_12_bus,emb_13_bus,emb_14_bus,emb_15_bus,emb_16_bus,emb_17_bus,emb_18_bus,emb_19_bus,emb_20_bus,emb_21_bus,emb_22_bus,emb_23_bus,emb_24_bus,emb_25_bus,emb_26_bus,emb_27_bus,emb_28_bus,emb_29_bus,emb_30_bus,emb_31_bus,emb_32_bus,emb_33_bus,emb_34_bus,emb_35_bus,emb_36_bus,emb_37_bus,emb_38_bus,emb_39_bus,emb_40_bus,emb_41_bus,emb_42_bus,emb_43_bus,emb_44_bus,emb_45_bus,emb_46_bus,emb_47_bus,emb_48_bus,emb_49_bus,emb_50_bus,emb_51_bus,emb_52_bus,emb_53_bus,emb_54_bus,emb_55_bus,emb_56_bus,emb_57_bus,emb_58_bus,emb_59_bus,emb_60_bus,emb_61_bus,emb_62_bus,emb_63_bus,emb_64_bus,emb_65_bus,emb_66_bus,emb_67_bus,emb_68_bus,emb_69_bus,emb_70_bus,emb_71_bus,emb_72_bus,emb_73_bus,emb_74_bus,emb_75_bus,emb_76_bus,emb_77_bus,emb_78_bus,emb_79_bus,emb_80_bus,emb_81_bus,emb_82_bus,emb_83_bus,emb_84_bus,emb_85_bus,emb_86_bus,emb_87_bus,emb_88_bus,emb_89_bus,emb_90_bus,emb_91_bus,emb_92_bus,emb_93_bus,emb_94_bus,emb_95_bus,emb_96_bus,emb_97_bus,emb_98_bus,emb_99_bus,emb_100_bus,emb_101_bus,emb_102_bus,emb_103_bus,emb_104_bus,emb_105_bus,emb_106_bus,emb_107_bus,emb_108_bus,emb_109_bus,emb_110_bus,emb_111_bus,emb_112_bus,emb_113_bus,emb_114_bus,emb_115_bus,emb_116_bus,emb_117_bus,emb_118_bus,emb_119_bus,emb_120_bus,emb_121_bus,emb_122_bus,emb_123_bus,emb_124_bus,emb_125_bus,emb_126_bus,emb_127_bus,emb_0_user,emb_1_user,emb_2_user,emb_3_user,emb_4_user,emb_5_user,emb_6_user,emb_7_user,emb_8_user,emb_9_user,emb_10_user,emb_11_user,emb_12_user,emb_13_user,emb_14_user,emb_15_user,emb_16_user,emb_17_user,emb_18_user,emb_19_user,emb_20_user,emb_21_user,emb_22_user,emb_23_user,emb_24_user,emb_25_user,emb_26_user,emb_27_user,emb_28_user,emb_29_user,emb_30_user,emb_31_user,emb_32_user,emb_33_user,emb_34_user,emb_35_user,emb_36_user,emb_37_user,emb_38_user,emb_39_user,emb_40_user,emb_41_user,emb_42_user,emb_43_user,emb_44_user,emb_45_user,emb_46_user,emb_47_user,emb_48_user,emb_49_user,emb_50_user,emb_51_user,emb_52_user,emb_53_user,emb_54_user,emb_55_user,emb_56_user,emb_57_user,emb_58_user,emb_59_user,emb_60_user,emb_61_user,emb_62_user,emb_63_user,emb_64_user,emb_65_user,emb_66_user,emb_67_user,emb_68_user,emb_69_user,emb_70_user,emb_71_user,emb_72_user,emb_73_user,emb_74_user,emb_75_user,emb_76_user,emb_77_user,emb_78_user,emb_79_user,emb_80_user,emb_81_user,emb_82_user,emb_83_user,emb_84_user,emb_85_user,emb_86_user,emb_87_user,emb_88_user,emb_89_user,emb_90_user,emb_91_user,emb_92_user,emb_93_user,emb_94_user,emb_95_user,emb_96_user,emb_97_user,emb_98_user,emb_99_user,emb_100_user,emb_101_user,emb_102_user,emb_103_user,emb_104_user,emb_105_user,emb_106_user,emb_107_user,emb_108_user,emb_109_user,emb_110_user,emb_111_user,emb_112_user,emb_113_user,emb_114_user,emb_115_user,emb_116_user,emb_117_user,emb_118_user,emb_119_user,emb_120_user,emb_121_user,emb_122_user,emb_123_user,emb_124_user,emb_125_user,emb_126_user,emb_127_user,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb

In [203]:
X, y = train_data.iloc[:, 1:], train_data.iloc[:, 0]

In [204]:
def objective_xgb(trial, data=X, target=y):
    """
    Objective function to tune an `XGBRegressor` model.
    """
    X_t, X_v, y_t, y_v = train_test_split(data, target, test_size=0.2, random_state=42)
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 300, 950),
        'reg_alpha': trial.suggest_float("reg_alpha", 1e-8, 100.0),
        'reg_lambda': trial.suggest_float("reg_lambda", 1e-8, 100.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0, step=0.1),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0, log=True),
        'max_depth': trial.suggest_int("max_depth", 5, 20),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'min_child_weight': trial.suggest_int("min_child_weight", 1, 300),
    }
    params["tree_method"] = "gpu_hist"
    params["predictor"] = "gpu_predictor"

    model = XGBRegressor(
        booster="gbtree",
        random_state=42,
        **params
    )
    model.fit(X_t, y_t)
    y_p = model.predict(X_v)
    rmse = mean_squared_error(y_v, y_p, squared=False)
    return rmse

In [ ]:
import optuna
study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=700)
study_xgb.best_params

[I 2022-12-06 00:25:29,169] A new study created in memory with name: no-name-dcf85541-1013-4713-8ddc-ff7a8de921c7


[00:25:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-12-06 00:28:50,468] Trial 1 finished with value: 1.0487263613016522 and parameters: {'n_estimators': 347, 'reg_alpha': 84.05843414827581, 'reg_lambda': 18.240618319174477, 'subsample': 0.5, 'learning_rate': 0.3983516185737862, 'max_depth': 11, 'colsample_bytree': 0.7933354796341269, 'min_child_weight': 62}. Best is trial 0 with value: 1.000903409021931.


[00:28:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-12-06 00:30:59,583] Trial 2 finished with value: 0.9760393196299091 and parameters: {'n_estimators': 473, 'reg_alpha': 88.41697807305184, 'reg_lambda': 59.417452015504715, 'subsample': 0.6, 'learning_rate': 0.016890560123989384, 'max_depth': 12, 'colsample_bytree': 0.8357928793982213, 'min_child_weight': 234}. Best is trial 2 with value: 0.9760393196299091.


...

[11:56:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-12-06 11:59:01,470] Trial 316 finished with value: 0.9735086929625053 and parameters: {'n_estimators': 916, 'reg_alpha': 65.16332488417275, 'reg_lambda': 95.15196180920469, 'subsample': 0.9, 'learning_rate': 0.015237862290534702, 'max_depth': 20, 'colsample_bytree': 0.9413055176228707, 'min_child_weight': 281}. Best is trial 96 with value: 0.9732051397134271.


[11:59:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-12-06 12:01:07,715] Trial 317 finished with value: 0.9738756210884603 and parameters: {'n_estimators': 930, 'reg_alpha': 60.93315042872983, 'reg_lambda': 92.1834241664782, 'subsample': 0.9, 'learning_rate': 0.019207227884649163, 'max_depth': 19, 'colsample_bytree': 0.9223345963373584, 'min_child_weight': 294}. Best is trial 96 with value: 0.9732051397134271.


[12:01:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [164]:
'''params = {'n_estimators': 936, 
          'reg_alpha': 41.966295719532596, 
          'reg_lambda': 63.69213525247807, 
          'subsample': 1.0, 
          'learning_rate': 0.015751233309040207, 
          'max_depth': 17, 
          'colsample_bytree': 0.7016995662147052, 
          'min_child_weight': 297}'''
params = {
    'n_estimators': 939, 
    'reg_alpha': 17.636094931978263, 
    'reg_lambda': 96.35830238072398, 
    'subsample': 0.9, 
    'learning_rate': 0.012699002442535388, 
    'max_depth': 13, 
    'colsample_bytree': 0.6909573774280149, 
    'min_child_weight': 84,
    'gpu_id': 1}
#500 best
'''params = {'n_estimators': 825,
 'reg_alpha': 47.872462235791964,
 'reg_lambda': 35.090392800972275,
 'subsample': 0.9,
 'learning_rate': 0.016420196193260114,
 'max_depth': 14,
 'colsample_bytree': 0.6697195454807059,
 'min_child_weight': 203,
 'gpu_id': 1}'''
params["tree_method"] = "gpu_hist"
params["predictor"] = "gpu_predictor"

min_rmse = 1.5
K = KFold(n_splits=10, random_state=42, shuffle=True)
for i, (idx_t, idx_v) in enumerate(K.split(X)):
    X_t, X_v = X.iloc[idx_t], X.iloc[idx_v]
    y_t, y_v = y.iloc[idx_t], y.iloc[idx_v]

    model = XGBRegressor(booster="gbtree", random_state=42, **params)
    
    model.fit(X_t, y_t)
    y_pred = model.predict(X_v)
    rmse = (mean_squared_error(y_pred, y_v) ** 0.5)
    print("Val RMSE: ", rmse)
    pickle.dump(model, open("./model_" + str(i) + "_comb_business.pkl", 'wb'))

[23:00:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Val RMSE:  0.9833983079868849
[23:02:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Val RMSE:  0.9722340223192141
[23:03:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Val RMSE:  0.9795498581913461
[23:05:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Val RMSE:  0.9770425327078582
[23:07:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Val RMSE:  0.9832685140619668
[23:09:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Val RMSE:  0.9770777432472378
[23:10:55] WARNING: /workspace/src/objective/regression_ob

In [188]:
df_val = pd.read_csv('/data/chaoyu/data553/yelp_val.csv')
df_val = df_val.rename(columns={'stars': 'target'})

In [189]:
#ALL
val_data = df_val.merge(embeddings, left_on='business_id', right_on='id', how='left')
val_data = val_data.merge(embeddings, left_on='user_id', right_on='id', how='left', suffixes=('_bus', '_user'))
val_data = val_data.merge(business_features, on='business_id', how='left')
val_data = val_data.merge(user_features, on='user_id', how='left')
val_data.drop(['user_id', 'business_id', 'id_bus', 'id_user'], axis=1, inplace=True)

#User
#val_data = df_val.merge(embeddings, left_on='user_id', right_on='id', how='left')
#val_data = val_data.merge(business_features, on='business_id', how='left')
#val_data = val_data.merge(user_features, on='user_id', how='left')
#val_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)

#Business
#val_data = df_val.merge(embeddings, left_on='business_id', right_on='id', how='left')
#val_data = val_data.merge(business_features, on='business_id', how='left')
#val_data = val_data.merge(user_features, on='user_id', how='left')
#val_data.drop(['user_id', 'business_id', 'id'], axis=1, inplace=True)

#Null
#val_data = df_val.merge(business_features, on='business_id', how='left')
#val_data = val_data.merge(user_features, on='user_id', how='left')
#val_data.drop(['user_id', 'business_id'], axis=1, inplace=True)
val_data

,target,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,emb_10_bus,emb_11_bus,emb_12_bus,emb_13_bus,emb_14_bus,emb_15_bus,emb_16_bus,emb_17_bus,emb_18_bus,emb_19_bus,emb_20_bus,emb_21_bus,emb_22_bus,emb_23_bus,emb_24_bus,emb_25_bus,emb_26_bus,emb_27_bus,emb_28_bus,emb_29_bus,emb_30_bus,emb_31_bus,emb_32_bus,emb_33_bus,emb_34_bus,emb_35_bus,emb_36_bus,emb_37_bus,emb_38_bus,emb_39_bus,emb_40_bus,emb_41_bus,emb_42_bus,emb_43_bus,emb_44_bus,emb_45_bus,emb_46_bus,emb_47_bus,emb_48_bus,emb_49_bus,emb_50_bus,emb_51_bus,emb_52_bus,emb_53_bus,emb_54_bus,emb_55_bus,emb_56_bus,emb_57_bus,emb_58_bus,emb_59_bus,emb_60_bus,emb_61_bus,emb_62_bus,emb_63_bus,emb_64_bus,emb_65_bus,emb_66_bus,emb_67_bus,emb_68_bus,emb_69_bus,emb_70_bus,emb_71_bus,emb_72_bus,emb_73_bus,emb_74_bus,emb_75_bus,emb_76_bus,emb_77_bus,emb_78_bus,emb_79_bus,emb_80_bus,emb_81_bus,emb_82_bus,emb_83_bus,emb_84_bus,emb_85_bus,emb_86_bus,emb_87_bus,emb_88_bus,emb_89_bus,emb_90_bus,emb_91_bus,emb_92_bus,emb_93_bus,emb_94_bus,emb_95_bus,emb_96_bus,emb_97_bus,emb_98_bus,emb_99_bus,emb_100_bus,emb_101_bus,emb_102_bus,emb_103_bus,emb_104_bus,emb_105_bus,emb_106_bus,emb_107_bus,emb_108_bus,emb_109_bus,emb_110_bus,emb_111_bus,emb_112_bus,emb_113_bus,emb_114_bus,emb_115_bus,emb_116_bus,emb_117_bus,emb_118_bus,emb_119_bus,emb_120_bus,emb_121_bus,emb_122_bus,emb_123_bus,emb_124_bus,emb_125_bus,emb_126_bus,emb_127_bus,emb_0_user,emb_1_user,emb_2_user,emb_3_user,emb_4_user,emb_5_user,emb_6_user,emb_7_user,emb_8_user,emb_9_user,emb_10_user,emb_11_user,emb_12_user,emb_13_user,emb_14_user,emb_15_user,emb_16_user,emb_17_user,emb_18_user,emb_19_user,emb_20_user,emb_21_user,emb_22_user,emb_23_user,emb_24_user,emb_25_user,emb_26_user,emb_27_user,emb_28_user,emb_29_user,emb_30_user,emb_31_user,emb_32_user,emb_33_user,emb_34_user,emb_35_user,emb_36_user,emb_37_user,emb_38_user,emb_39_user,emb_40_user,emb_41_user,emb_42_user,emb_43_user,emb_44_user,emb_45_user,emb_46_user,emb_47_user,emb_48_user,emb_49_user,emb_50_user,emb_51_user,emb_52_user,emb_53_user,emb_54_user,emb_55_user,emb_56_user,emb_57_user,emb_58_user,emb_59_user,emb_60_user,emb_61_user,emb_62_user,emb_63_user,emb_64_user,emb_65_user,emb_66_user,emb_67_user,emb_68_user,emb_69_user,emb_70_user,emb_71_user,emb_72_user,emb_73_user,emb_74_user,emb_75_user,emb_76_user,emb_77_user,emb_78_user,emb_79_user,emb_80_user,emb_81_user,emb_82_user,emb_83_user,emb_84_user,emb_85_user,emb_86_user,emb_87_user,emb_88_user,emb_89_user,emb_90_user,emb_91_user,emb_92_user,emb_93_user,emb_94_user,emb_95_user,emb_96_user,emb_97_user,emb_98_user,emb_99_user,emb_100_user,emb_101_user,emb_102_user,emb_103_user,emb_104_user,emb_105_user,emb_106_user,emb_107_user,emb_108_user,emb_109_user,emb_110_user,emb_111_user,emb_112_user,emb_113_user,emb_114_user,emb_115_user,emb_116_user,emb_117_user,emb_118_user,emb_119_user,emb_120_user,emb_121_user,emb_122_user,emb_123_user,emb_124_user,emb_125_user,emb_126_user,emb_127_user,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb

In [190]:
val_data.describe()

,target,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,emb_10_bus,emb_11_bus,emb_12_bus,emb_13_bus,emb_14_bus,emb_15_bus,emb_16_bus,emb_17_bus,emb_18_bus,emb_19_bus,emb_20_bus,emb_21_bus,emb_22_bus,emb_23_bus,emb_24_bus,emb_25_bus,emb_26_bus,emb_27_bus,emb_28_bus,emb_29_bus,emb_30_bus,emb_31_bus,emb_32_bus,emb_33_bus,emb_34_bus,emb_35_bus,emb_36_bus,emb_37_bus,emb_38_bus,emb_39_bus,emb_40_bus,emb_41_bus,emb_42_bus,emb_43_bus,emb_44_bus,emb_45_bus,emb_46_bus,emb_47_bus,emb_48_bus,emb_49_bus,emb_50_bus,emb_51_bus,emb_52_bus,emb_53_bus,emb_54_bus,emb_55_bus,emb_56_bus,emb_57_bus,emb_58_bus,emb_59_bus,emb_60_bus,emb_61_bus,emb_62_bus,emb_63_bus,emb_64_bus,emb_65_bus,emb_66_bus,emb_67_bus,emb_68_bus,emb_69_bus,emb_70_bus,emb_71_bus,emb_72_bus,emb_73_bus,emb_74_bus,emb_75_bus,emb_76_bus,emb_77_bus,emb_78_bus,emb_79_bus,emb_80_bus,emb_81_bus,emb_82_bus,emb_83_bus,emb_84_bus,emb_85_bus,emb_86_bus,emb_87_bus,emb_88_bus,emb_89_bus,emb_90_bus,emb_91_bus,emb_92_bus,emb_93_bus,emb_94_bus,emb_95_bus,emb_96_bus,emb_97_bus,emb_98_bus,emb_99_bus,emb_100_bus,emb_101_bus,emb_102_bus,emb_103_bus,emb_104_bus,emb_105_bus,emb_106_bus,emb_107_bus,emb_108_bus,emb_109_bus,emb_110_bus,emb_111_bus,emb_112_bus,emb_113_bus,emb_114_bus,emb_115_bus,emb_116_bus,emb_117_bus,emb_118_bus,emb_119_bus,emb_120_bus,emb_121_bus,emb_122_bus,emb_123_bus,emb_124_bus,emb_125_bus,emb_126_bus,emb_127_bus,emb_0_user,emb_1_user,emb_2_user,emb_3_user,emb_4_user,emb_5_user,emb_6_user,emb_7_user,emb_8_user,emb_9_user,emb_10_user,emb_11_user,emb_12_user,emb_13_user,emb_14_user,emb_15_user,emb_16_user,emb_17_user,emb_18_user,emb_19_user,emb_20_user,emb_21_user,emb_22_user,emb_23_user,emb_24_user,emb_25_user,emb_26_user,emb_27_user,emb_28_user,emb_29_user,emb_30_user,emb_31_user,emb_32_user,emb_33_user,emb_34_user,emb_35_user,emb_36_user,emb_37_user,emb_38_user,emb_39_user,emb_40_user,emb_41_user,emb_42_user,emb_43_user,emb_44_user,emb_45_user,emb_46_user,emb_47_user,emb_48_user,emb_49_user,emb_50_user,emb_51_user,emb_52_user,emb_53_user,emb_54_user,emb_55_user,emb_56_user,emb_57_user,emb_58_user,emb_59_user,emb_60_user,emb_61_user,emb_62_user,emb_63_user,emb_64_user,emb_65_user,emb_66_user,emb_67_user,emb_68_user,emb_69_user,emb_70_user,emb_71_user,emb_72_user,emb_73_user,emb_74_user,emb_75_user,emb_76_user,emb_77_user,emb_78_user,emb_79_user,emb_80_user,emb_81_user,emb_82_user,emb_83_user,emb_84_user,emb_85_user,emb_86_user,emb_87_user,emb_88_user,emb_89_user,emb_90_user,emb_91_user,emb_92_user,emb_93_user,emb_94_user,emb_95_user,emb_96_user,emb_97_user,emb_98_user,emb_99_user,emb_100_user,emb_101_user,emb_102_user,emb_103_user,emb_104_user,emb_105_user,emb_106_user,emb_107_user,emb_108_user,emb_109_user,emb_110_user,emb_111_user,emb_112_user,emb_113_user,emb_114_user,emb_115_user,emb_116_user,emb_117_user,emb_118_user,emb_119_user,emb_120_user,emb_121_user,emb_122_user,emb_123_user,emb_124_user,emb_125_user,emb_126_user,emb_127_user,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb

In [191]:
val_dense = ['stars', 'review_count_b', 'review_count_u', 'useful', 'funny', 'cool', 'fans', 
               'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 
               'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 
               'compliment_writer', 'compliment_photos']
mms = MinMaxScaler(feature_range=(0,1))
val_data[val_dense] = mms.fit_transform(val_data[val_dense])
val_data


,target,emb_0_bus,emb_1_bus,emb_2_bus,emb_3_bus,emb_4_bus,emb_5_bus,emb_6_bus,emb_7_bus,emb_8_bus,emb_9_bus,emb_10_bus,emb_11_bus,emb_12_bus,emb_13_bus,emb_14_bus,emb_15_bus,emb_16_bus,emb_17_bus,emb_18_bus,emb_19_bus,emb_20_bus,emb_21_bus,emb_22_bus,emb_23_bus,emb_24_bus,emb_25_bus,emb_26_bus,emb_27_bus,emb_28_bus,emb_29_bus,emb_30_bus,emb_31_bus,emb_32_bus,emb_33_bus,emb_34_bus,emb_35_bus,emb_36_bus,emb_37_bus,emb_38_bus,emb_39_bus,emb_40_bus,emb_41_bus,emb_42_bus,emb_43_bus,emb_44_bus,emb_45_bus,emb_46_bus,emb_47_bus,emb_48_bus,emb_49_bus,emb_50_bus,emb_51_bus,emb_52_bus,emb_53_bus,emb_54_bus,emb_55_bus,emb_56_bus,emb_57_bus,emb_58_bus,emb_59_bus,emb_60_bus,emb_61_bus,emb_62_bus,emb_63_bus,emb_64_bus,emb_65_bus,emb_66_bus,emb_67_bus,emb_68_bus,emb_69_bus,emb_70_bus,emb_71_bus,emb_72_bus,emb_73_bus,emb_74_bus,emb_75_bus,emb_76_bus,emb_77_bus,emb_78_bus,emb_79_bus,emb_80_bus,emb_81_bus,emb_82_bus,emb_83_bus,emb_84_bus,emb_85_bus,emb_86_bus,emb_87_bus,emb_88_bus,emb_89_bus,emb_90_bus,emb_91_bus,emb_92_bus,emb_93_bus,emb_94_bus,emb_95_bus,emb_96_bus,emb_97_bus,emb_98_bus,emb_99_bus,emb_100_bus,emb_101_bus,emb_102_bus,emb_103_bus,emb_104_bus,emb_105_bus,emb_106_bus,emb_107_bus,emb_108_bus,emb_109_bus,emb_110_bus,emb_111_bus,emb_112_bus,emb_113_bus,emb_114_bus,emb_115_bus,emb_116_bus,emb_117_bus,emb_118_bus,emb_119_bus,emb_120_bus,emb_121_bus,emb_122_bus,emb_123_bus,emb_124_bus,emb_125_bus,emb_126_bus,emb_127_bus,emb_0_user,emb_1_user,emb_2_user,emb_3_user,emb_4_user,emb_5_user,emb_6_user,emb_7_user,emb_8_user,emb_9_user,emb_10_user,emb_11_user,emb_12_user,emb_13_user,emb_14_user,emb_15_user,emb_16_user,emb_17_user,emb_18_user,emb_19_user,emb_20_user,emb_21_user,emb_22_user,emb_23_user,emb_24_user,emb_25_user,emb_26_user,emb_27_user,emb_28_user,emb_29_user,emb_30_user,emb_31_user,emb_32_user,emb_33_user,emb_34_user,emb_35_user,emb_36_user,emb_37_user,emb_38_user,emb_39_user,emb_40_user,emb_41_user,emb_42_user,emb_43_user,emb_44_user,emb_45_user,emb_46_user,emb_47_user,emb_48_user,emb_49_user,emb_50_user,emb_51_user,emb_52_user,emb_53_user,emb_54_user,emb_55_user,emb_56_user,emb_57_user,emb_58_user,emb_59_user,emb_60_user,emb_61_user,emb_62_user,emb_63_user,emb_64_user,emb_65_user,emb_66_user,emb_67_user,emb_68_user,emb_69_user,emb_70_user,emb_71_user,emb_72_user,emb_73_user,emb_74_user,emb_75_user,emb_76_user,emb_77_user,emb_78_user,emb_79_user,emb_80_user,emb_81_user,emb_82_user,emb_83_user,emb_84_user,emb_85_user,emb_86_user,emb_87_user,emb_88_user,emb_89_user,emb_90_user,emb_91_user,emb_92_user,emb_93_user,emb_94_user,emb_95_user,emb_96_user,emb_97_user,emb_98_user,emb_99_user,emb_100_user,emb_101_user,emb_102_user,emb_103_user,emb_104_user,emb_105_user,emb_106_user,emb_107_user,emb_108_user,emb_109_user,emb_110_user,emb_111_user,emb_112_user,emb_113_user,emb_114_user,emb_115_user,emb_116_user,emb_117_user,emb_118_user,emb_119_user,emb_120_user,emb_121_user,emb_122_user,emb_123_user,emb_124_user,emb_125_user,emb_126_user,emb_127_user,state,is_open,stars,review_count_b,postal_code,emb_city_0,emb_city_1,emb_city_2,emb_city_3,emb_city_4,emb_city_5,emb_city_6,emb_city_7,emb_city_8,emb_city_9,emb_city_10,emb_city_11,emb_city_12,emb_city_13,emb_city_14,emb_city_15,emb_city_16,emb_city_17,emb_city_18,emb_city_19,emb_city_20,emb_city_21,emb_city_22,emb_city_23,emb_city_24,emb_city_25,emb_city_26,emb_city_27,emb_city_28,emb_city_29,emb_city_30,emb_city_31,emb_city_32,emb_city_33,emb_city_34,emb_city_35,emb_city_36,emb_city_37,emb_city_38,emb_city_39,emb_city_40,emb_city_41,emb_city_42,emb_city_43,emb_city_44,emb_city_45,emb_city_46,emb_city_47,emb_city_48,emb_city_49,emb_city_50,emb_city_51,emb_city_52,emb_city_53,emb_city_54,emb_city_55,emb_city_56,emb_city_57,emb_city_58,emb_city_59,emb_city_60,emb_city_61,emb_city_62,emb_city_63,emb_city_64,emb_city_65,emb_city_66,emb_city_67,emb_city_68,emb_city_69,emb_city_70,emb_city_71,emb_city_72,emb_city_73,emb_city_74,emb_city_75,emb_city_76,emb_city_77,emb_city_78,emb_city_79,emb_city_80,emb

In [192]:
X_val, y_val = val_data.iloc[:, 1:], val_data.iloc[:, 0]

In [198]:
#LINE Val RMSE: 0.9736
#Val RMSE: 0.9715
#User Val RMSE: 0.9731
#Business Val RMSE: 0.9831
#Null Val RMSE: 0.9780
#Comb RMSE: 0.8565
#Comb User Val RMSE: 0.8768
#Comb Business Val RMSE: 0.9055
#Comb Null Val RMSE: 0.9220
res = []
for i in range(10):
    model = pickle.load(open("./model_" + str(i) + ".pkl", 'rb'))
    res.append(list(model.predict(X_val)))
res = np.array(res)
y_pred = np.mean(res, axis=0)
mse = mean_squared_error(y_pred, y_val)
print("Val RMSE: %.16f" % (mse ** 0.5))

[00:23:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:23:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re